# Working with imbalanded classification data

Author: [Thomas Moreau](https://tommoral.github.io/)

In [ ]:
# Global import
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.base import clone
import matplotlib.pyplot as plt
from IPython.display import display, HTML

%matplotlib inline

For this notebook, we will need the imabalanced learn package (doc available here: https://imbalanced-learn.readthedocs.io/en/stable/index.html)

In [ ]:
!pip install -U imbalanced-learn

# The problem of class imbalanced in supervised classification

## Highlight the issue of working with imbalanced classes

To illustrate the issue of imbalance learn, we will use the `adult_census` dataset.  
A complete description of the data can be found [here](https://www.openml.org/d/1590). Basically, the task is to predict whether an individual will have an annual income of more than 50K dollars based on his personal information.

In [ ]:
df = pd.read_csv("https://www.openml.org/data/get_csv/1595261/adult-census.csv")

In [ ]:
df.head()

In [ ]:
display(HTML("<h3>Categorial variable</h3>"))
display(df.describe(exclude=np.number))
display(HTML("<h3>Numerical variable</h3>"))
display(df.describe(exclude=np.object))

We can analyze the target column, here called `class` which we will be using to train a model.

In [ ]:
from collections import Counter

In [ ]:
counter = Counter(df['class'])
counter

By looking at the classes, we can observe that there is 2 classes: `' >50K'` and `' <=50K'`.
In addition, we can see that there is a difference regarding the classes frequencies. We can compute the balancing ratio.

In [ ]:
counter[' <=50K'] / counter[' >50K']

We can check what it would imply if we are not careful when evaluating our model. We will train a `DummyClassifier` which will not predict by learning anything from the data but rather predict the most frequent class in the dataset.

In [ ]:
# Get the data. We extract the labels
# We drop the fnlwgt feature which is not informative in
# this context (see data description for more info).
target_name = "class"
target = df[target_name].to_numpy()
data = df.drop(columns=[target_name, "fnlwgt"])

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(data, target)

Let's verify that the classifier always outputs the most frequent class.

In [ ]:
dummy_clf.predict([[0]])

In [ ]:
y_pred = dummy_clf.predict(data)

Now, let's see what it implies regarding the default metric with scikit-learn classifier.

In [ ]:
print(f"The accuracy of the most-frequent model is "
      f"{dummy_clf.score(data, target):.3f}")

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print(f"The accuracy of the default model is "
      f"{accuracy_score(target, y_pred):.3f}")

We can here see the main issue: **Unlike with balanced learning, the naive classification level is not `0.5`**.  
It is necessary to take into account the class unbalance to help the model learn something meaningful.

## Step 1: Use one or several informative metrics to detect the issue

Instead of using the `accuracy_score`, one could use the `balanced_accuracy_score`.
*Slight issue:* not everyone agrees on the definition of the metric.

The `balanced_accuracy` avoids inflated performance estimates on imbalanced datasets:
* If the conventional accuracy is above chance only because the classifier takes advantage of an imbalanced test set, then the balanced accuracy, as appropriate, will drop to $\frac{1}{n_{classes}}$.
* If the classifier performs equally well on either class, this term reduces to the conventional accuracy (*i.e.*, the number of correct predictions divided by the total number of predictions).




*Details:* It is the raw accuracy where each sample is weighted according to the inverse prevalence of its true class. Given predicted $\widehat y_i$ for sample $i$, balanced accuracy is defined as:
$$
    \texttt{balanced-accuracy}(y, \widehat y) = \frac{1}{\sum w_i} \sum_i w_i 1\{\widehat y_i = y_i\}
$$
with $w_i = \frac{1}{\sum_j 1\{y_j = y_i\}}$, the frequency of class $y_i$ in the dataset.

In [ ]:
from sklearn.metrics import balanced_accuracy_score

print(f"The balanced accuracy of the default model is "
      f"{balanced_accuracy_score(target, y_pred):.3f}")

Then, we can have a look at the confusion matrix to a real idea of what is happening.

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(target, y_pred)

In [ ]:
cm_df = pd.DataFrame(
    confusion_matrix(target, y_pred),
    columns=dummy_clf.classes_,
    index=dummy_clf.classes_
)
sns.set(font_scale=1.4) # for label size
sns.heatmap(
    cm_df, annot=True, annot_kws={"size": 16},
    cmap='Oranges'
)

plt.xlim(0, 2)
plt.ylim(0, 2)

As we have seen in the first part of the lab, the confusion matrix can be summarized using several metrics

* precision and recall
* sensitivity and specificity
* area under the roc curve (ROC-AUC)

NB: sensitivity = recall
 
If we take as a positive class `' >50K'`, we get:

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

precision = precision_score(
    target, y_pred, pos_label=' >50K'
)
recall = recall_score(
    target, y_pred, pos_label=' >50K'
)

print(f"The recall of the dummy model is "
      f"{recall}")
print(f"The precision of the dummy model is "
      f"{precision}")

In [ ]:
from imblearn.metrics import sensitivity_score
from imblearn.metrics import specificity_score

sensitivity = sensitivity_score(
    target, y_pred, pos_label=' >50K')
specificity = specificity_score(
   target, y_pred, pos_label=' >50K')

print(f"The sensitivity of the dummy model is "
      f"{sensitivity}")
print(f"The specificity of the dummy model is "
      f"{specificity}")

## Step 2: How to solve the issue during `fit`

In this part, we will see how we can adapt the model to cope with imbalanced dataset.
First, we split the data in a train and a test set. Then we review various way to handle imbalanced data.

In [ ]:
# split in train test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, target, random_state=0
)

In [ ]:
# Helper function to evaluate the different models
def evaluate_classifier(clf):
    name = getattr(clf, 'name', clf.__class__.__name__)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    y_pred = clf.predict(X_test)
    balanced_score = balanced_accuracy_score(y_test, y_pred)
    display(HTML(
        f"<h5>{name}</h5>"
        f"Test Accuracy: {score:7.2%} <br/>"
        f"Balanced accuracy: {balanced_score:7.2%} <br/>"
    ))

### 2.1 Baseline classifier

As we have seen before, we will compare the new classifiers with a dummy baseline which predict the most frequent label in the dataset. This baseline will highlight the improvement compared to naive strategies.

In [ ]:
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.name = "Most Frequent Classifier"

evaluate_classifier(dummy_clf)

### 2.2 Make use of the `class_weight` parameter to modify the training cost

A first class of methods rely on sample weights to correct the imbalance. The core idea here is to weight prediction mistakes on the minority class higher than mistakes on the most common class.

#### In linear model - modifying the cost function

In `scikit-learn`, some estimators have a `class_weight` parameter that permits to do this. The idea is that the ERM is changed such that
$$
    \arg\min_\theta \frac{1}{\sum_i w_i} \sum_i w_i 1\{f_\theta(X_i) = y_i\}
$$
with weights $w_i = \frac{n}{kn_i}$ with $n$ the total number of samples, $k$ the number of classes and $n_i$ the number of samples from class $y_i$. This effectively rebalance the training in learning both from positive and negative examples.



In [ ]:
binary_encoding_columns = ['sex']
one_hot_encoding_columns = [
    'workclass', 'education', 'marital-status',
    'occupation', 'relationship', 'race', 'native-country'
]
scaling_columns = [
    'age', 'education-num', 'hours-per-week',
    'capital-gain', 'capital-loss'
]

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

preprocessor_lr = ColumnTransformer([
    ('binary-encoder', OrdinalEncoder(), binary_encoding_columns),
    ('one-hot-encoder', OneHotEncoder(handle_unknown='ignore'),
     one_hot_encoding_columns),
    ('standard-scaler', StandardScaler(), scaling_columns)
])
model_lr = make_pipeline(
    preprocessor_lr,
    LogisticRegression(solver='lbfgs', max_iter=1000)
)
model_lr.name = "Logistic Regression"

In [ ]:
evaluate_classifier(model_lr)

We can set the `class_weight='balanced'` uses the values of `y` to automatically adjust weights inversely proportional to class frequencies in the input data.

In [ ]:
model_lr_balanced = clone(model_lr)
model_lr_balanced.set_params(
    logisticregression__class_weight='balanced')
model_lr_balanced.name = "Logistic Regression with balanced weights"
evaluate_classifier(model_lr_balanced)

#### In tree-based model - modifying the purity criterion

In tree based models, the `class_weight` option is used to chose on the splits. Indeed, the purity criterion (which is minimize for to chose the split) is computed using these weights. In the leaf, the weights are used to compute the class to output.

In [ ]:
ordinal_encoding_columns = ['workclass', 'education', 'marital-status',
                            'occupation', 'relationship', 'sex',
                            'race', 'native-country']
scaling_columns = ['age', 'education-num', 'hours-per-week',
                   'capital-gain', 'capital-loss']

In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestClassifier

preprocessor_rf = ColumnTransformer([
    ('ordinal-encoder', OrdinalEncoder(), ordinal_encoding_columns),
    ('standard-scaler', FunctionTransformer(validate=False), scaling_columns)
])
model_rf = make_pipeline(
    preprocessor_rf,
    RandomForestClassifier(n_estimators=100, random_state=42)
)
model_rf.name = "Random Forest"

In [ ]:
evaluate_classifier(model_rf)

In [ ]:
model_rf_balanced = clone(model_rf)
model_rf_balanced.set_params(
    randomforestclassifier__class_weight='balanced'
)
model_rf_balanced.name = "Balanced Random Forest"

evaluate_classifier(model_rf_balanced)

For ensemble models, `class_weight` can also take value `balanced_subsample`. This option is equivalent to the `'balanced'` one except that the weigths are computed directly for the bootstrap sample of each tree instead of weights computed globally.

In [ ]:
model_rf_subbalanced = clone(model_rf)
model_rf_subbalanced.set_params(
    randomforestclassifier__class_weight='balanced_subsample'
)
model_rf_subbalanced.name = "Balanced Subsample Random Forest"

evaluate_classifier(model_rf_subbalanced)

### 2.3 Resample the training set to have balanced classes

A second option to learn on unbalanced data is to reweight the classes by sampling a new training set with balanced class. This can be done by either subsampling, oversampling or more complicated scheme demonstrated in this section.

#### Random under-sampling during training

In [ ]:
from imblearn.pipeline import make_pipeline as make_pipeline_imblearn
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
model_lr_undersampled = make_pipeline_imblearn(
    preprocessor_lr,
    RandomUnderSampler(),
    LogisticRegression(solver='lbfgs', max_iter=1000)
)
model_lr_undersampled.name = "Logistic Regression from rebalanced undersampled data"

In [ ]:
evaluate_classifier(model_lr_undersampled)

#### Random over-sampling during training

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
model_lr_oversampled = make_pipeline_imblearn(
    preprocessor_lr,
    RandomOverSampler(),
    LogisticRegression(solver='lbfgs', max_iter=1000)
)
model_lr_oversampled.name = "Logistic Regression from rebalanced oversampled data"

In [ ]:
evaluate_classifier(model_lr_oversampled)

#### More fancy methods


There exists some more fancy methods to re-balance the dataset. For instance the SMOTE method where extra points are generated by creatinig synthetic points for the minority class. See more info on the [original paper](https://arxiv.org/pdf/1106.1813.pdf) or in this [blog post](http://rikunert.com/SMOTE_explained).

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
model_lr_smote = make_pipeline_imblearn(
    preprocessor_lr,
    SMOTE(),
    LogisticRegression(solver='lbfgs', max_iter=1000)
)
model_lr_smote.name = "Logistic Regression from SMOTE sampled data"

In [ ]:
evaluate_classifier(model_lr_smote)

You can look in [imbalanced-learn documentation](https://imbalanced-learn.readthedocs.io/en/stable/api.html#module-imblearn.over_sampling) for more sampling strategies.

### 2.4 Used balanced algorithms: `BalancedRandomForest` and `BalancedBaggingClassifier` 

Instead of just sampling the training set to rebalance the classes, it is also possible to used _balanced_ classifier to fit the unbalanced dataset. The core idea is to use ensemble techniques with specific boostrap sampling strategies that make sure that each bootstrap sample is balanced.

#### Example of `BalancedRandomForestClassifier`

Here, a random forest is learn on the full dataset. Each tree is constructed using a balanced sub-sampled of the dataset.  
This idea has been proposed by [Chen et al. (2004)](https://statistics.berkeley.edu/sites/default/files/tech-reports/666.pdf).

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier

model_balanced_rf = make_pipeline(
    preprocessor_rf,
    BalancedRandomForestClassifier(
        n_estimators=100, random_state=42
    )
)
model_balanced_rf.name = "Balanced Random Forest"

In [ ]:
evaluate_classifier(model_balanced_rf)

#### Example of `BalancedBaggingClassifier`

In ensemble classifiers, bagging methods build several estimators on different randomly selected subset of data. In scikit-learn, this classifier is named `BaggingClassifier`. However, this classifier does not allow to balance each subset of data. Therefore, when training on imbalanced data set, this classifier will favor the majority classes.

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier

In [ ]:
model_bagging = make_pipeline(
    preprocessor_rf,
    BaggingClassifier(
        base_estimator=HistGradientBoostingClassifier(),
        n_estimators=10, random_state=42
    )
)
model_bagging.name = "Bagging Model"

In [ ]:
evaluate_classifier(model_bagging)

`BalancedBaggingClassifier `allows to resample each subset of data before to train each estimator of the ensemble. In short, it combines the output of an `EasyEnsemble` sampler with an ensemble of classifiers (i.e. `BaggingClassifier`). Therefore, `BalancedBaggingClassifier `takes the same parameters than the scikit-learn `BaggingClassifier`. Additionally, there is two additional parameters, sampling_strategy and replacement to control the behaviour of the random under-sampler.

In [ ]:
from imblearn.ensemble import BalancedBaggingClassifier
model_balanced_bagging = make_pipeline(
    preprocessor_rf,
    BalancedBaggingClassifier(
        base_estimator=HistGradientBoostingClassifier(),
        n_estimators=10, random_state=42
    )
)
model_balanced_bagging.name = "Balanced Bagging Model"

In [ ]:
evaluate_classifier(model_balanced_bagging)